In [1]:
import rasterio
import numpy as np
import os
import tools

file_ends = ['channels', '5haStreams']
paths = [
    '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5_0.5m_elev.tif',
    '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/5haStreams_0.5m_elev.tif'
]

resos = [20, 40, 80, 160]
catchment_no = np.arange(1, 23)
exclude = [11, 17, 18, 19]
catchs = np.setdiff1d(catchment_no, exclude)
nodata_val = -9999  # consistent nodata value

# -----------------------------
# Step 1: Reproject/resample 0.5 m elevations and mask with coarse raster
# -----------------------------
for elev_05m_fp, file in zip(paths, file_ends):
    for catch in catchs:
        for reso in resos:
            outdir = f"/Users/jpnousu/Krycklan_GIS_data/{reso}m/C{catch}"
            os.makedirs(outdir, exist_ok=True)

            # Reference coarse raster for alignment
            ref_raster_fp = f"{outdir}/{file}.asc"
            cmask_fp = f"{outdir}/cmask.asc"

            # Output file for resampled elevation
            reproj_fp = os.path.join(outdir, f"{file}_elev.asc")

            # Resample 0.5 m raster to coarse resolution
            tools.reproj_match(
                infile=elev_05m_fp,
                match=ref_raster_fp,
                outfile=reproj_fp,
                resampling_method='min',
                save_in='asc'
            )

            # -----------------------------
            # Apply coarse mask: keep only pixels that are valid in the elevation raster
            # -----------------------------
            with rasterio.open(reproj_fp) as elev_src:
                elev_data = elev_src.read(1)
                elev_meta = elev_src.meta.copy()
                elev_nodata = elev_meta.get('nodata', nodata_val)

            # Open coarse mask
            with rasterio.open(ref_raster_fp) as mask_src:
                mask_data = mask_src.read(1)
                mask_meta = mask_src.meta.copy()
                mask_nodata = mask_meta.get('nodata', 0)

            # Update coarse mask so it matches elevation raster
            mask_data[elev_data == elev_nodata] = mask_nodata

            # Save updated coarse mask
            with rasterio.open(ref_raster_fp, 'w', **mask_meta) as dst:
                dst.write(mask_data, 1)

            # Optionally, you can also enforce the nodata in elevation raster
            elev_data[mask_data == mask_nodata] = nodata_val
            elev_meta.update(nodata=nodata_val)
            with rasterio.open(reproj_fp, 'w', **elev_meta) as dst:
                dst.write(elev_data.astype(np.float32), 1)

# -----------------------------
# Step 2: Calculate stream depths
# -----------------------------
for raster in file_ends:
    for catch in catchs:
        for reso in resos:
            stream_ele_path = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/C{catch}/{raster}_elev.asc'
            dem_path = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/C{catch}/dem_clip.asc'
            stream_depth_path = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/C{catch}/{raster}_depth.asc'

            # Open elevation raster
            with rasterio.open(stream_ele_path) as stream_src:
                stream_ele = stream_src.read(1)
                stream_meta = stream_src.meta.copy()

            # Open DEM
            with rasterio.open(dem_path) as dem_src:
                dem_data = dem_src.read(1)

            # Create masks
            dem_mask = (np.isfinite(dem_data)) & (dem_data != -9999)
            stream_mask = (stream_ele != -9999)

            # Calculate depth where both are valid
            stream_depth = np.where(dem_mask & stream_mask, stream_ele - dem_data, -9999)

            # Apply thresholds
            valid = stream_depth != -9999
            stream_depth[valid & (stream_depth > 0)] = 0
            stream_depth[valid & (stream_depth < -15)] = -15

            # Save depth raster
            stream_meta.update(dtype=rasterio.float32, count=1, nodata=-9999)
            with rasterio.open(stream_depth_path, 'w', **stream_meta) as dst:
                dst.write(stream_depth.astype(rasterio.float32), 1)

Coregistered to shape: 69 41 
 Affine | 20.00, 0.00, 732306.00|
| 0.00,-20.00, 7134870.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 35 21 
 Affine | 40.00, 0.00, 732286.00|
| 0.00,-40.00, 7134870.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 18 11 
 Affine | 80.00, 0.00, 732246.00|
| 0.00,-80.00, 7134870.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 9 6 
 Affine | 160.00, 0.00, 732166.00|
| 0.00,-160.00, 7134870.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 39 30 
 Affine | 20.00, 0.00, 730786.00|
| 0.00,-20.00, 7134570.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 20 16 
 Affine | 40.00, 0.00, 730766.00|
| 0.00,-40.00, 7134590.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 11 9 
 Affine | 80.00, 0.00, 730726.00|
| 0.00,-80.00, 7134630.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 6 5 
 Affine | 160.00, 0.00, 730726.00|
| 0.00,-160.00, 7134710.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 13 21 
 Affine | 20.00, 0.00, 731326.00|
| 0.00,-20.00, 7134490.00|
| 0.00, 0.00, 1.00|
Co